In [1]:
%load_ext bigdata
%hive_start
%timeout 300

In [2]:
%%hive

DROP TABLE IF EXISTS t0;
CREATE TABLE t0 (
    c1 STRING,
    c2 ARRAY<CHAR(1)>, 
    c3 MAP<STRING, INT>
    )
    ROW FORMAT DELIMITED 
        FIELDS TERMINATED BY '\t'
        COLLECTION ITEMS TERMINATED BY ','
        MAP KEYS TERMINATED BY '#'
        LINES TERMINATED BY '\n';
LOAD DATA LOCAL INPATH 'data.tsv' INTO TABLE t0;

DROP TABLE IF EXISTS t0;
OK
Time taken: 17.54 seconds
CREATE TABLE t0 (
    c1 STRING,
    c2 ARRAY<CHAR(1)>, 
    c3 MAP<STRING, INT>
    )
    ROW FORMAT DELIMITED 
        FIELDS TERMINATED BY '\t'
        COLLECTION ITEMS TERMINATED BY ','
        MAP KEYS TERMINATED BY '#'
        LINES TERMINATED BY '\n';
OK
Time taken: 1.587 seconds
LOAD DATA LOCAL INPATH 'data.tsv' INTO TABLE t0;
Loading data to table default.t0
OK
Time taken: 2.012 seconds


In [4]:
%%hive
DROP TABLE IF EXISTS respuesta;
CREATE TABLE respuesta
AS
    SELECT 
        letra,
        key,
        COUNT(value)
    FROM 
        t0
        LATERAL VIEW explode(c2) adTable AS letra
        LATERAL VIEW explode(c3) adTable AS key,value
    GROUP BY letra,key; 

INSERT OVERWRITE LOCAL DIRECTORY 'output'
ROW FORMAT DELIMITED FIELDS TERMINATED BY ','
STORED AS TEXTFILE
SELECT * FROM respuesta;


DROP TABLE IF EXISTS respuesta;
OK
Time taken: 0.195 seconds
CREATE TABLE respuesta
AS
    SELECT 
        letra,
        key,
        COUNT(value)
    FROM 
        t0
        LATERAL VIEW explode(c2) adTable AS letra
        LATERAL VIEW explode(c3) adTable AS key,value
    GROUP BY letra,key; 
Query ID = root_20200130230301_8400f17e-7ed2-4560-8cfa-fb8d51035a73
Total jobs = 1
Launching Job 1 out of 1
Number of reduce tasks not specified. Estimated from input data size: 1
In order to change the average load for a reducer (in bytes):
  set hive.exec.reducers.bytes.per.reducer=<number>
In order to limit the maximum number of reducers:
  set hive.exec.reducers.max=<number>
In order to set a constant number of reducers:
  set mapreduce.job.reduces=<number>
Job running in-process (local Hadoop)
2020-01-30 23:03:03,046 Stage-1 map = 100%,  reduce = 100%
Ended Job = job_local131132326_0003
Moving data to directory file:/user/hive/warehouse/respuesta
MapReduce Jobs Launched: 
Stage-Stage-1:  

In [5]:
%hive_quit